In [27]:
import pandas as pd
from scipy.fft import fft, ifft,fftfreq
from matplotlib import pyplot as plt
import os
import librosa
from scipy.signal import hann,blackman
import numpy as np
from scipy.signal import hilbert, chirp

In [2]:
DATA_DIR = '/mnt/disks/extra_data/kaggle/ventilator_prediction/'
R_MAP = {5: 0, 50: 1, 20: 2}
C_MAP = {20: 0, 50: 1, 10: 2}

In [6]:
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))
# train.breath_id = train.breath_id.map(dict(zip(train.breath_id.unique().tolist(),range(train.breath_id.nunique()))))
# test.breath_id = train.breath_id.map(dict(zip(train.breath_id.unique().tolist(),range(train.breath_id.nunique()))))

In [7]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [18]:
train = train.groupby('breath_id').head(40)

In [44]:
temp = train[train.breath_id==125743]

In [43]:
train.breath_id.unique()

array([     1,      2,      3, ..., 125743, 125745, 125749])

In [16]:
# librosa.stft(n_fft=80,win_length=20,y=temp['u_in'].values)

In [ ]:
train.groupby('breath_id')

In [29]:
# %%time
# w = hann(40+1)
# ffta = lambda x: np.abs(fft(np.append(x.values,x.values[0]))[:40])
# ffta.__name__ = 'ffta'

# fftw = lambda x: np.abs(fft(np.append(x.values,x.values[0])*w)[:40])
# fftw.__name__ = 'fftw'

# train['fft_u_in'] = train.groupby('breath_id')['u_in'].transform(ffta)
# train['fft_u_in_w'] = train.groupby('breath_id')['u_in'].transform(fftw)
train['analytical'] = train.groupby('breath_id')['u_in'].transform(hilbert)
train['envelope'] = np.abs(train['analytical'])
train['phase'] = np.angle(train['analytical'])
train['unwrapped_phase'] = train.groupby('breath_id')['phase'].transform(np.unwrap)
train['phase_shift1'] = train.groupby('breath_id')['unwrapped_phase'].shift(1).astype(np.float32)
train['IF'] = train['unwrapped_phase'] - train['phase_shift1'].astype(np.float32)

/home/sahil_okcredit_in/miniconda3/envs/torch-py3.8/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:1097: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype, copy=True)


In [ ]:
temp[['u_in','fft_u_in','fft_u_in_w']].plot()

In [30]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,fft_u_in,fft_u_in_w,analytical,envelope,phase,unwrapped_phase,phase_shift1,IF
0,1,1,20,50,0.000000,0.083334,0,5.837492,663.678061,398.764745,0.083334,0.083334,0.0,0.0,NaN,NaN
1,2,1,20,50,0.033652,18.383041,0,5.907794,283.160288,250.164926,18.383041,18.383041,0.0,0.0,0.0,0.0
2,3,1,20,50,0.067514,22.509278,0,7.876254,114.792768,28.882860,22.509278,22.509278,0.0,0.0,0.0,0.0
3,4,1,20,50,0.101542,22.808822,0,11.742872,21.281852,24.593310,22.808822,22.808822,0.0,0.0,0.0,0.0
4,5,1,20,50,0.135756,25.355850,0,12.234987,41.132863,14.712721,25.355850,25.355850,0.0,0.0,0.0,0.0


In [32]:
train.head(20)

,id,breath_id,R,C,time_step,u_in,u_out,pressure,fft_u_in,fft_u_in_w,analytical,envelope,phase,unwrapped_phase,phase_shift1,IF
0,1,1,20,50,0.000000,0.083334,0,5.837492,663.678061,398.764745,0.083334,0.083334,0.0,0.0,NaN,NaN
1,2,1,20,50,0.033652,18.383041,0,5.907794,283.160288,250.164926,18.383041,18.383041,0.0,0.0,0.0,0.0
2,3,1,20,50,0.067514,22.509278,0,7.876254,114.792768,28.882860,22.509278,22.509278,0.0,0.0,0.0,0.0
3,4,1,20,50,0.101542,22.808822,0,11.742872,21.281852,24.593310,22.808822,22.808822,0.0,0.0,0.0,0.0
4,5,1,20,50,0.135756,25.355850,0,12.234987,41.132863,14.712721,25.355850,25.355850,0.0,0.0,0.0,0.0
5,6,1,20,50,0.169698,27.259866,0,12.867706,42.516541,7.733396,27.259866,27.259866,0.0,0.0,0.0,0.0
6,7,1,20,50,0.203708,27.127486,0,14.695562,22.749459,8.293207,27.127486,27.127486,0.0,0.0,0.0,0.0
7,8,1,20,50,0.237723,26.807732,0,15.890699,7.092306,8.363611,26.807732,26.807732,0.0,0.0,0.0,0.0
8,9,1,20,50,0.271776,27.864715,0,15.539188,26.905698,8.332176,27.864715,27.864715,0.0,0.0,0.0,0.0
9,10,1,20,50,0.305732,28.313036,0,15.750094,20.945069,5.691678,28.313036,28.313036,0.0,0.0,0.0,0.0
